In [1]:
%matplotlib inline

import json
import requests
import zipfile
import pandas as pd
import geopandas as gpd
import polyline as pl
import googlemaps
from itertools import permutations, combinations
from shapely.geometry import LineString

# Data prep for flows tutorial

This notebook compiles steps required to compile street and straight distances between all bike share stations within the city of San Francisco and obtain the number of trips that were taken over the period of September 2015 to August 2016. This relies on the following two files:

* Stations

In [2]:
url_stns = 'http://www.bayareabikeshare.com/stations/json'

* Trips

In [3]:
url_trips = 'https://s3.amazonaws.com/babs-open-data/babs_open_data_year_3.zip'

## Distances

* Reading the stations file. Keep only those in San Francisco:

In [4]:
js = json.loads(requests.get(url_stns).text)

stns = pd.DataFrame(js['stationBeanList'])

stns = stns.loc[stns['landMark']=='San Francisco', :]
stns.head(3)

,altitude,availableBikes,availableDocks,city,id,is_renting,landMark,lastCommunicationTime,latitude,location,longitude,postalCode,stAddress1,stAddress2,stationName,status,statusKey,statusValue,testStation,totalDocks
14,,10,9,San Francisco,39,True,San Francisco,2017-03-12 15:54:33,37.783871,San Francisco,-122.408433,,Powell Street BART,Market,Powell Street BART,IN_SERVICE,1,In Service,False,19
15,,3,12,San Francisco,41,True,San Francisco,2017-03-12 15:54:02,37.795001,,-122.399970,,Clay at Battery,Clay Street,Clay at Battery,IN_SERVICE,1,In Service,False,15
16,,3,12,San Francisco,42,True,San Francisco,2017-03-12 15:50:51,37.797280,,-122.398436,,Davis at Jackson,Davis Street,Davis at Jackson,IN_SERVICE,1,In Service,False,15


* Compiling all possible destinations from station to station.

In [5]:
od_ids = pd.DataFrame([(i[0], i[1], str(i[0])+'-'+str(i[1])) for i in
             permutations(stns['id'], 2)],
                      columns=['orig', 'dest', 'flow_id'])
od_ids['line'] = None

* Pull lines + distance from Google

In [6]:
key = open('key').readline().strip('\n')
gmaps = googlemaps.Client(key=key)

In [7]:
%%time
for id, pair in od_ids.iterrows():
    xy1 = stns.loc[\
            stns['id']==pair['orig'], ['latitude', 'longitude']\
                  ].iloc[0].tolist()
    xy2 = stns.loc[\
            stns['id']==pair['dest'], ['latitude', 'longitude']\
                  ].iloc[0].tolist()
    drs = gmaps.directions(xy1, xy2, mode='bicycling')
    line = drs[0]['overview_polyline']['points']
    od_ids.loc[id, 'line'] = line

CPU times: user 22.3 s, sys: 1.18 s, total: 23.4 s
Wall time: 3min 21s


In [8]:
# Save just in case
od_ids.to_csv('lines.csv')

* Encode trips as `shapely` line objects decoding them with [`polyline`](https://pypi.python.org/pypi/polyline/1.3.2).

In [9]:
def rearrange(l):
    '''
    Swap latitude for longitude so it conforms
    to XY as `LineString` expects
    '''
    return list(map(lambda t: t[::-1], l))

In [10]:
od_ids['geometry'] = od_ids['line'].apply(\
                    lambda l: LineString(\
                               rearrange(pl.decode(l))\
                                        )\
                                             )

* Turn the table into a `GeoDataFrame`

In [11]:
od = gpd.GeoDataFrame(od_ids.drop('geometry', axis=1), \
                      geometry=od_ids['geometry'], \
                      crs={'init' :'epsg:4326'})

* Project to the NAD83 / California Albers projection ([`EPSG:3310`](http://epsg.io/3310)), expressed in metres.

In [12]:
od = od.to_crs(epsg=3310)

### Distances

* Obtain street distances

In [13]:
od['street_dist'] = od.length

* Obtain straight distances

In [14]:
def straight_dist(line):
    xys = line.coords
    stl = LineString([xys[0], xys[-1]])
    return stl.length

In [15]:
od['straight_dist'] = od['geometry'].apply(straight_dist)

### Elevation

In [16]:
key = open('key_ele').readline().strip('\n')
gmaps = googlemaps.Client(key=key)

* Retriever of elevation every 100 metres approx

In [92]:
def elevator(row):
    # Get N. of samples
    n = int( pd.np.round( row['geometry'].length / 100. ) ) \
        + 1
    # Call Google
    try:
        ele = gmaps.elevation_along_path(row['line'], n)
        # Extract elevations
        path = pd.DataFrame(ele)['elevation']
        # Process (min, max, mean, average slope)
        ups = []
        downs = []
        steps = path[1:].shape[0]
        if steps == 1:
            steps += 1
        for i in range(1, steps):
            step = path.iloc[i] - path.iloc[i-1]
            if step > 0:
                ups.append(step)
            elif step < 0:
                downs.append(step)
            else:
                pass
        total_up = sum(ups)
        total_down = sum(downs)
        summary =  pd.Series({'total_up': total_up, \
                              'total_down': -total_down,\
                              'flow_id': row['flow_id'],\
                              'profile': path.values
                             })
    except:
        summary =  pd.Series({'total_up': None, \
                              'total_down': None,\
                              'flow_id': row['flow_id'],\
                              'profile': None
                             })
    return summary

* Pull from Google

In [58]:
%%time
elevations = od.apply(elevator, axis=1)

CPU times: user 15.6 s, sys: 1.19 s, total: 16.8 s
Wall time: 4min 10s


* Join in to `od`

In [105]:
od = od.join(elevations.set_index('flow_id'), on='flow_id')

---

* Rearrange for convenience

In [107]:
od = od.set_index('flow_id')\
       .loc[:, ['geometry', 'orig', 'dest', \
                'street_dist', 'straight_dist', \
                'total_down', 'total_up']]
       
od.head()

,geometry,orig,dest,street_dist,straight_dist,total_down,total_up
flow_id,,,,,,,
39-41,LINESTRING (-211794.3346253044 -23163.40897385...,39,41,1804.114975,1452.201024,11.205753,4.698162
39-42,LINESTRING (-211794.3346253044 -23163.40897385...,39,42,2069.155725,1734.860910,10.290236,2.897886
39-45,LINESTRING (-211794.3346253044 -23163.40897385...,39,45,1747.992761,1255.349224,11.015596,4.593927
39-46,LINESTRING (-211794.3346253044 -23163.40897385...,39,46,1490.836113,1323.303409,3.511543,5.038044
39-47,LINESTRING (-211794.3346253044 -23163.40897385...,39,47,769.918884,715.688981,0.000000,3.282495


## Trips

For this, we assume the trips file is downloaded to the same folder as the notebook. If that's not the case, the following cells download it and unpack it.

---

* Download file

In [402]:
%%time
r = requests.get(url_trips)
with open('trips.zip', 'wb') as f:
    f.write(r.content)

CPU times: user 2.96 s, sys: 2.41 s, total: 5.37 s
Wall time: 1min 46s


* Extract only the file we need from the downloaded `zip` file:

In [403]:
zf = zipfile.ZipFile('trips.zip')
zf.extract('201608_trip_data.csv', './')

'201608_trip_data.csv'

---

* Read the trips in

In [108]:
trips = pd.read_csv('201608_trip_data.csv')

* ID each trip as above

In [109]:
trips['flow_id'] = trips['Start Terminal'].astype(str) \
                   + '-' + \
                   trips['End Terminal'].astype(str)

* Aggregate trips by flow ID and join to flows table *and* fill remaining pairs with zero

In [110]:
od['trips'] = pd.DataFrame({'trips': trips.groupby('flow_id').size()})
od['trips'] = od['trips'].fillna(0)

In [111]:
od.head()

,geometry,orig,dest,street_dist,straight_dist,total_down,total_up,trips
flow_id,,,,,,,,
39-41,LINESTRING (-211794.3346253044 -23163.40897385...,39,41,1804.114975,1452.201024,11.205753,4.698162,68.0
39-42,LINESTRING (-211794.3346253044 -23163.40897385...,39,42,2069.155725,1734.860910,10.290236,2.897886,29.0
39-45,LINESTRING (-211794.3346253044 -23163.40897385...,39,45,1747.992761,1255.349224,11.015596,4.593927,50.0
39-46,LINESTRING (-211794.3346253044 -23163.40897385...,39,46,1490.836113,1323.303409,3.511543,5.038044,163.0
39-47,LINESTRING (-211794.3346253044 -23163.40897385...,39,47,769.918884,715.688981,0.000000,3.282495,73.0


* Reproject to lon/lat

In [112]:
od = od.to_crs(epsg=4326)

* Write out as a `GeoJSON`

In [114]:
od.to_file('flows.geojson', driver='GeoJSON')